In [1]:
from ckks_package.ckks import *

In [2]:
N = 2**15  # Ring degree
n = 2**2  # Number of slots
L = 20  # Maximal level
q0 = 2**40  # Smallest modulus
delta = 2**33  # Scaling factor

# The grouping parameter for bootstrapping is s. It lies between 1 and
# log(n, 2). The smaller s, the faster the bootstrapping, but the more levels
# are consumed.
s = log(n, 2)

CKKS.config(N, n, L, q0, delta)
CKKS.key_gen()
CKKS.config_bootstrap(CKKS.sk, s)

The CKKS configuration is done!

The key generation is done!
Estimated security: 2^(80.0).

Encoding relevant vectors as polynomials...
Generating matrices required for CoeffToSlot and SlotToCoeff...
Encoding these matrices as polynomials...
Generating missing switching keys...
The bootstrapping configuration is done!



In [3]:
# Creating two random complex vectors

complex_vectors = [
    np.array([randint(-3, 3) + 1j * randint(-3, 3) for _ in range(n)])
    for _ in range(2)
]
complex_vectors

[array([-3.-3.j, -1.+0.j, -3.-2.j,  3.-2.j]),
 array([-2.-1.j, -1.-2.j, -0.-1.j,  3.-1.j])]

In [4]:
# Encoding as polynomials

plaintext_polys = [CKKS.encode(z) for z in complex_vectors]
plaintext_polys

[- 8589934592 + 6433451618X^4096 - 16703502750X^8192 - 11563693771X^12288 - 15032385536X^16384 + 4308435740X^20480 + 7592501250X^24576 - 140999689X^28672 mod(2^700),
 - 2664822913X^4096 - 4555500750X^8192 - 8757871169X^12288 - 10737418240X^16384 + 10401483995X^20480 + 7592501250X^24576 - 1303209465X^28672 mod(2^700)]

In [5]:
# Encrypting

ciphertexts = [CKKS.enc_poly_with_sk(pt, CKKS.sk) for pt in plaintext_polys]
ciphertexts

[A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20),
 A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20)]

In [6]:
# Checking correctness

for i in range(2):
    pt = ciphertexts[i].dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[-3.-3.00000000e+00j -1.+1.89902871e-10j -3.-2.00000000e+00j
  3.-2.00000000e+00j]
[-2.00000000e+00-1.j -1.00000000e+00-2.j -1.28159172e-09-1.j
  3.00000000e+00-1.j]


In [7]:
# Homomorphic operations

ct_add = ciphertexts[0] + ciphertexts[1]
ct_mul = (
    ciphertexts[0] @ ciphertexts[1]
)  # Polynomial multiplication followed by a rescaling operation
print(ct_add)
print(ct_mul)

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20)
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^19 (level 19 out of 20)


In [8]:
# Checking correctness

for ct in [ct_add, ct_mul]:
    pt = ct.dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[-5.-4.j -2.-2.j -3.-3.j  6.-3.j]
[ 3.+9.00000001j  1.+2.j         -2.+3.j          7.-9.j        ]


In [9]:
# Bootstrapping

ct = ciphertexts[0] % q0  # Project to the lowest level l = 0
print(ct)

ct_boot = ct.bootstrap(s)
print(ct_boot)

pt = ct_boot.dec_to_poly(CKKS.sk)
print(CKKS.decode(pt))

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^0 (level 0 out of 20)
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^2 (level 2 out of 20)
[-2.9495004 -2.96712801j -1.02260971+0.05950124j -3.04292846-2.00479175j
  3.01665234-2.03458368j]
